#### Test notebook that generates .tif files and stores in /results folder. Use **python37** kernel.

In [1]:
import os
import sys
import rasterio
import pyproj
import zipfile
import tempfile
import random
import requests
import time

from rasterio.windows import Window
from rasterio import Affine
from shapely.geometry import Polygon
from shapely.ops import transform
from pathlib import Path
from datetime import datetime

In [2]:
sys.version

'3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 02:25:08) \n[GCC 7.5.0]'

In [3]:
NB_USER = os.getenv('NB_USER')
print(NB_USER)

jovyan


In [4]:
tetleha = [(
              36.63013458251953,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.87273390600898
            ), ]

forest = [(
              36.747894287109375,
              50.008400954338185
            ),
            (
              36.88556671142578,
              49.874282751903394
            ),
            (
              37.00366973876953,
              49.92647191927194
            ),
            (
              36.84711456298828,
              50.03707625174409
            ),
            (
              36.747894287109375,
              50.008400954338185
            ),
          ]
kharkiv = [(
              36.066741943359375,
              50.03773778732018
            ),
            (
              36.25556945800781,
              49.87295517274949
            ),
            (
              36.46568298339844,
              49.96447253142292
            ),
            (
              36.31874084472656,
              50.072565780723615
            ),
            (
              36.066741943359375,
              50.03773778732018
            ),
          ]

In [5]:
def _str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start="2020-01-01T00:00:00", end=datetime.today().strftime('%Y-%m-%dT%H:%M:%S')):
    return _str_time_prop(start, end, '%Y-%m-%dT%H:%M:%S', random.random())

In [6]:
def crop(input_path, output_path, polygon, name):
    with rasterio.open(input_path) as src:
        x_min, y_min, x_max, y_max = polygon.bounds
        
        affine = Affine(src.transform[0],
                        src.transform[1],
                        x_min,
                        src.transform[3],
                        src.transform[4],
                        y_max)

        row_min, col_max = rasterio.transform.rowcol(src.transform, x_max, y_max)
        row_max, col_min = rasterio.transform.rowcol(src.transform, x_min, y_min, op=round, precision=6)
        row_min = row_min + 1
        row_max = row_max + 1
        
        write_window = Window.from_slices([row_min, row_max], [col_min, col_max])
        raster = src.read(window=write_window)
        kwargs = src.meta.copy()
        kwargs.update({
            'height': write_window.height,
            'width': write_window.width,
            "transform": affine,
            'driver': 'GTiff'
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            if random.choice([True, False]):
                print(f"Randomly selected to add name {name}")
                dst.update_tags(name=name)
            if random.choice([True, False]):             
                start_date = random_date()
                end_date = random_date(start_date)
                print(f"Randomly selected to add start_date: {start_date} and end_date: {end_date}")   
                dst.update_tags(start_date=start_date, end_date=end_date)
            dst.write(raster)

In [7]:
def convert(input_path, output_path, format='GTiff'):
    with rasterio.open(input_path) as src:
        raster = src.read()
        crs = str(src.crs)
                
        kwargs = src.meta.copy()
        kwargs.update({'driver': format})
        
        
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(raster)
        return crs

In [8]:
def to_crs(poly, target, current='EPSG:4326'):
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [9]:
def load_tile(dir_, url):
    tile_filename = url.split("/")[-1]
    path = f"{dir_}/{tile_filename}"
    if os.path.exists(path):
        print(f"Tile {path} already exists! Using it...")
        return path
    
    print(f"Loading tile: {url}")
    
    with requests.Session() as session:
        response = session.get(url)
        if response.status_code == 200:
            with open(path, 'wb') as file:
                file.write(response.content)
        else:
            raise ValueError(f"Cannot load tile {url}. Check and rerun!")
    print(f"Tile loaded and stored: {path}")
    return path

In [10]:
def generate(dir_, tile_path, aoi, location_name):
    
    stem = Path(tile_path).stem
    tif_path = f"{dir_}/{stem}.tif"    
    target_crs = convert(tile_path, tif_path)
    aoi = to_crs(Polygon(aoi), target_crs)

    temp_output_path = f"/home/{NB_USER}/work/results/{stem}_{location_name}.tif.temp"
    
    crop(tif_path, temp_output_path, aoi, location_name)
    
    print(f"Generated temp file: {temp_output_path}")
    output_path = temp_output_path[:-5]
    print(f"Rename temp file: {temp_output_path} to {output_path}")
    os.rename(temp_output_path, output_path)

In [11]:
dir_ = f"/home/{NB_USER}/work/satellite_imagery"

TILE_URL = "https://storage.googleapis.com/gcp-public-data-sentinel-2/tiles/36/U/YA/" \
           "S2B_MSIL1C_20200705T083559_N0209_R064_T36UYA_20200705T114142.SAFE/GRANULE/" \
           "L1C_T36UYA_A017393_20200705T083602/IMG_DATA/T36UYA_20200705T083559_TCI.jp2"

### main execution cell

In [ ]:
tile_path = load_tile(dir_, TILE_URL)
with tempfile.TemporaryDirectory(dir=dir_) as tmpdirname:
    print('Сreated temporary directory for calculations:', tmpdirname)
        
    locations = {'tetleha': tetleha, 'forest': forest, 'kharkiv': kharkiv}
    name = random.choice(list(locations.keys()))
    aoi = locations[name]
    
    generate(tmpdirname, tile_path, aoi, name)

print("Execution finished")